In [1]:
import pandas as pd
import string
import nltk
import emoji
import joblib
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
!pip install openpyxl
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Load the dataset from the full path (update if needed)
df = pd.read_excel(r"C:\Users\DELL\OneDrive\Desktop\sample_project\emotion\dataset.xlsx")

# Drop empty rows
df.dropna(inplace=True)

# Preview the data
df.head()


,text,label
0,"Honestly, really engaging panel discussions. l...",positive
1,Impressed by the tech setup. Very professional.,positive
2,"Not bad, just feels a bit disorganized.",neutral
3,Fantastic networking opportunities today!,positive
4,Sound quality in breakout rooms was awful. Tha...,negative


In [4]:
def preprocess(text):
    text = emoji.demojize(text)  # Convert emojis to text
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters & digits
    words = text.split()
    words = [word for word in words if word not in stop_words]  # Remove stopwords
    return ' '.join(words)


In [5]:
import re
df['clean_text'] = df['text'].astype(str).apply(preprocess)
df[['text', 'clean_text']].head()


,text,clean_text
0,"Honestly, really engaging panel discussions. l...",honestly really engaging panel discussions loved
1,Impressed by the tech setup. Very professional.,impressed tech setup professional
2,"Not bad, just feels a bit disorganized.",bad feels bit disorganized
3,Fantastic networking opportunities today!,fantastic networking opportunities today
4,Sound quality in breakout rooms was awful. Tha...,sound quality breakout rooms awful thanks


In [6]:
# Normalize labels
df['label'] = df['label'].str.lower()

# Optional: Merge similar labels
df['label'] = df['label'].replace({
    'positive': 'positive',
    'good': 'positive',
    'negative': 'negative',
    'bad': 'negative',
    'neutral': 'neutral',
    'Neutral': 'neutral'
})


In [7]:
X = df['clean_text']
y = df['label']

vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)


In [8]:
model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

In [9]:
y_pred = model.predict(X_test)

# Accuracy and detailed performance
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))


✅ Accuracy: 0.9477611940298507

📊 Classification Report:
               precision    recall  f1-score   support

    negative       0.89      0.98      0.93        41
     neutral       1.00      0.89      0.94        35
    positive       0.97      0.97      0.97        58

    accuracy                           0.95       134
   macro avg       0.95      0.94      0.95       134
weighted avg       0.95      0.95      0.95       134



In [10]:
joblib.dump(model, "sentiment_model.joblib")
joblib.dump(vectorizer, "vectorizer.joblib")
print("✅ Model and vectorizer saved to disk.")


✅ Model and vectorizer saved to disk.


In [11]:
# Reload if needed
model = joblib.load("sentiment_model.joblib")
vectorizer = joblib.load("vectorizer.joblib")

# Predict sentiment of a new sentence
def predict_sentiment(text):
    cleaned = preprocess(text)
    vec = vectorizer.transform([cleaned])
    return model.predict(vec)[0]

# Test prediction
print(predict_sentiment("The session was amazing! 🔥👏"))


positive
